In [2]:
import sys, os, time
%load_ext autoreload
%autoreload 2
import foundation as fd
import foundation.util as util
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
import torchvision
import h5py as hf
from html.parser import HTMLParser
import urllib
import urllib.request
from urllib.error import URLError, HTTPError
import requests
from extraction import *
import redis
import itertools

In [3]:
root = '/home/fleeb/workspace/ml_datasets/wiki/wikimedia/'
root = '/mnt/d/Users/anwan/Documents/wikigraph/wikimedia/'
root = '/mnt/c/Users/anwan/Documents/wikigraph/'

In [4]:
fname = 'enwiki-20181220-all-titles-in-ns0'

In [ ]:
with open(os.path.join(root, fname), 'r') as f:
    text = f.readlines()
text = text[1:]
text = [t[:-1].replace('_', ' ') for t in text]
len(text)

In [6]:
db = redis.Redis(port=6381)
rdb = redis.Redis(port=6381, db=3)
db.dbsize(), rdb.dbsize()

In [8]:
db.scard('redirs'), db.scard('articles')

(8569359, 5775900)

In [16]:
arts = [db.srandmember('articles') for _ in range(10)]
len(arts), arts

(10,
 [b'Conway Smith',
  b'Lawrence Newell',
  b'Geetingsville, Indiana',
  b'Kozub',
  b'Moira railway station (England)',
  b'Came glasswork',
  b'Bear Creek Ca\xc3\xb1on Park',
  b'Min Dawlya',
  b'Nodicoeloceratinae',
  b'Homer Judd'])

In [34]:
name = arts[2] #b'Artificial intelligence'#arts[0]
links = get_links(name)
links = [l['title'].encode() for l in links]
name, len(links)

(b'Geetingsville, Indiana', 66)

In [35]:
links

[b'Antioch, Clinton County, Indiana',
 b'Avery, Indiana',
 b'Beard, Indiana',
 b'Boyleston, Indiana',
 b'Cambria, Indiana',
 b'Center Township, Clinton County, Indiana',
 b'City',
 b'Civil township',
 b'Clinton County, Indiana',
 b'Colfax, Indiana',
 b'Countries of the world',
 b'County seat',
 b'Cyclone, Indiana',
 b'East Park, Indiana',
 b'Edna Mills, Indiana',
 b'Federal Information Processing Standard',
 b'Fickle, Indiana',
 b'Forest, Indiana',
 b'Forest Township, Clinton County, Indiana',
 b'Frankfort, Indiana',
 b'Geographic Names Information System',
 b'Geographic coordinate system',
 b'Hamilton, Clinton County, Indiana',
 b'Hillisburg, Indiana',
 b'Incorporated town',
 b'Indiana',
 b'International Standard Book Number',
 b'Jackson Township, Clinton County, Indiana',
 b'Jefferson, Indiana',
 b'Johnson Township, Clinton County, Indiana',
 b'Kilmore, Indiana',
 b'Kirklin, Indiana',
 b'Kirklin Township, Clinton County, Indiana',
 b'List of counties in Indiana',
 b'List of townships

In [36]:
for i, l in enumerate(links):
    if db.sismember('articles', l):
        print('art', l)
    elif db.sismember('redirs', l):
        print('redir', l)
    else:
        print('*failed', l)

art b'Antioch, Clinton County, Indiana'
art b'Avery, Indiana'
art b'Beard, Indiana'
art b'Boyleston, Indiana'
art b'Cambria, Indiana'
art b'Center Township, Clinton County, Indiana'
art b'City'
art b'Civil township'
art b'Clinton County, Indiana'
art b'Colfax, Indiana'
redir b'Countries of the world'
art b'County seat'
art b'Cyclone, Indiana'
*failed b'East Park, Indiana'
art b'Edna Mills, Indiana'
redir b'Federal Information Processing Standard'
art b'Fickle, Indiana'
art b'Forest, Indiana'
art b'Forest Township, Clinton County, Indiana'
art b'Frankfort, Indiana'
art b'Geographic Names Information System'
art b'Geographic coordinate system'
art b'Hamilton, Clinton County, Indiana'
art b'Hillisburg, Indiana'
art b'Incorporated town'
art b'Indiana'
art b'International Standard Book Number'
art b'Jackson Township, Clinton County, Indiana'
art b'Jefferson, Indiana'
art b'Johnson Township, Clinton County, Indiana'
art b'Kilmore, Indiana'
art b'Kirklin, Indiana'
art b'Kirklin Township, Clin

11583710

In [5]:
#redir = db.smembers('redirs')
redir = [db.srandmember('articles') for _ in range(70)] + [b'Jumping spider']
len(redir)

NameError: name 'db' is not defined

In [10]:
ITER = util.batches(arts, 50)

In [11]:
x = next(ITER)

In [ ]:
x[0]

In [15]:
DATA = get_links(x)
#DATA = get_all_redirs(b'Grapholitha subcorticana|Fao_festival|Zeiraphera_subcorticana|Salticidae')

In [16]:
DATA.keys()

dict_keys(['batchcomplete', 'query'])

In [17]:
DATA['query'].keys()

dict_keys(['redirects', 'pages'])

In [24]:
out = DATA['query']['redirects']
len(out)

36

In [31]:
#db.rem('test')
db.scard('redirs')

11087936

In [27]:
opts = set(x)
for o in out:
    assert o['from'].encode() in opts, str(o)

In [29]:
def work_fn(db, names, **unused):
    
    db.sadd('todo', *names)
    
def init_fn(**unused):
    out = {
        'db':redis.Redis(port=6381, db=0)
    }
    return out

In [30]:
shared = {}
def list_names():
    for r in util.batches(arts, n=50):
    #for i, n in enumerate(arts)
        yield {'names': r}
volatile = iter(list_names())

In [31]:
creator = util.Farmer(fn=work_fn, init_fn=init_fn, volatile_gen=volatile, num_workers=12, auto_dispatch=False, waiting=0)

In [32]:
creator._dispatch(len(arts)//50)

In [33]:
len(arts)//50

115518

In [34]:
inc = 0
for n in creator:
    inc += 1
    if inc % 10000 == 0:
        print(inc+1)

10001
20001
30001
40001
50001
60001
70001
80001
90001
100001
110001


In [36]:
del creator

In [35]:
db.scard('todo')

5775900

In [5]:
names = set(text)

In [6]:
'Exclamation_mark_(punctuation)' in text

False

In [439]:
'❕' in text

True

In [13]:
for name in names:
    break

In [14]:
name

'Thur_(disambiguation)'

In [ ]:
URL = "https://en.wikipedia.org/w/api.php"

PARAMS = {
    "action": "query",
    "format": "json",
    "list": "allpages",
    #'list': "alllinks",
    #'allimit': '10',
    #'titles': 'Exclamation mark',
    #'titles': 'Double_factorial|Artificial_intelligence|AI ethics',
    'prop': "links",
    #'redirects':True,
    #'alprop': 'ids|title',
    #"apfrom": "jungle", # continue
}

PARAMS = { # get links
    "action": "query",
    "format": "json",
    "titles": "Wesolowskana_lymphatica",
    #'titles': 'Jumping spider',
    "prop": "links",
    'redirects':True,
    'pllimit': 'max',
    'plnamespace': '0',
}

# PARAMS = {
#     'action': "query",
#     'format': "json",
#     'rdnamespace': '0',
#     #'titles': "Salticidae",
#     #'titles': "Jumping_spider",
#     'titles': 'Exclamation_mark',
#     'prop': "redirects",
#     'rdlimit':'10',
# }

PARAMS = {
    'action': "query",
    'format': "json",
    #'rdnamespace': '0',
    'titles': 'Zeiraphera_subcorticana',#'Fao_festival',
    'titles': 'Grapholitha subcorticana|Fao_festival|Zeiraphera_subcorticana|Salticidae',
    #'titles': 'Fao_festival',
    'prop': "redirects",
    #'rdlimit': 'max',
    'redirects': True,
}

PARAMS = {  # get links
    "action": "query",
    "format": "json",
    'titles': b'Grapholitha subcorticana',
    "prop": "links",
    'redirects': True,
    'pllimit': 'max',
    'plnamespace': '0',
}

R = requests.get(url=URL, params=PARAMS)
DATA = R.json()

In [ ]:
R.url

In [ ]:
DATA.keys()

In [ ]:
DATA['continue']

In [ ]:
DATA['query'].keys()

In [73]:
DATA['query']['redirects']

[{'from': 'Salticidae', 'to': 'Jumping spider'},
 {'from': 'Grapholitha subcorticana', 'to': 'Zeiraphera subcorticana'}]

In [74]:
DATA['query']['normalized']

[{'from': 'Fao_festival', 'to': 'Fao festival'},
 {'from': 'Zeiraphera_subcorticana', 'to': 'Zeiraphera subcorticana'}]

In [75]:
DATA['query']['pages']

{'276115': {'ns': 0,
  'pageid': 276115,
  'redirects': [{'ns': 0, 'pageid': 341051, 'title': 'Salticidae'},
   {'ns': 0, 'pageid': 4631934, 'title': 'Jumping spiders'},
   {'ns': 0, 'pageid': 6046836, 'title': 'Salticoidea'},
   {'ns': 0, 'pageid': 6198198, 'title': 'Attid spider'},
   {'ns': 0, 'pageid': 8573939, 'title': 'Jumping-spider'},
   {'ns': 0, 'pageid': 8718453, 'title': 'Attidae'},
   {'ns': 0, 'pageid': 19494313, 'title': 'Salticid'},
   {'ns': 0, 'pageid': 24177055, 'title': 'Jumping Spider'},
   {'ns': 0, 'pageid': 28179119, 'title': 'Saltids'}],
  'title': 'Jumping spider'},
 '34200083': {'ns': 0, 'pageid': 34200083, 'title': 'Fao festival'},
 '43656607': {'ns': 0,
  'pageid': 43656607,
  'redirects': [{'ns': 0,
    'pageid': 43656613,
    'title': 'Paedisca subcorticana'}],
  'title': 'Zeiraphera subcorticana'}}

In [297]:
'https://en.wikipedia.org/wiki/Salticidae'

'https://en.wikipedia.org/wiki/Salticidae'

In [9]:
db = redis.Redis()

In [10]:
db.dbsize()

3

In [11]:
db.keys()

[b'\xe2\x9d\x95\xe2\x9d\x95\xe2\x9d\x95', b'\xe2\x9d\x95', b'test']

In [12]:
db.set(10, 12)

True

In [13]:
db.keys()

[b'\xe2\x9d\x95\xe2\x9d\x95\xe2\x9d\x95', b'\xe2\x9d\x95', b'10', b'test']

In [14]:
db.get(10)

b'12'

In [18]:
str(10).encode()

b'10'

In [429]:
db.set('test','1')

True

In [427]:
db.shutdown()

In [430]:
db.set('❕', '❕❕')

True

In [19]:
'❕'.encode()

b'\xe2\x9d\x95'

In [431]:
db.keys()

[b'\xe2\x9d\x95', b'test']

In [432]:
db.get('❕')

b'\xe2\x9d\x95\xe2\x9d\x95'

In [20]:
db.sadd('x', '10')

1

In [21]:
db.sadd('x', '12')

1

In [22]:
db.smembers('x')

{b'10', b'12'}

In [26]:
db.spop('x')

In [434]:
db.hset('❕❕❕','❕', '❕❕')

1

In [438]:
db.hget('❕❕❕', '❕').decode()

'❕❕'

In [5]:
name = text[np.random.randint(len(text))]
name = 'Artificial intelligence'
name

'Artificial intelligence'

In [7]:
ID, title, redirs = get_redirs(name)
ID, title, len(redirs) if redirs is not None else None

(1164, 'Artificial intelligence', 56)

In [8]:
redirs

[{'pageid': 1268, 'ns': 0, 'title': 'AI'},
 {'pageid': 83979, 'ns': 0, 'title': 'Machine intelligence'},
 {'pageid': 87949, 'ns': 0, 'title': 'AI ethics'},
 {'pageid': 87959, 'ns': 0, 'title': 'AI implications'},
 {'pageid': 246362, 'ns': 0, 'title': 'Artificially intelligent'},
 {'pageid': 524509, 'ns': 0, 'title': 'Artificial Intelligence'},
 {'pageid': 638220, 'ns': 0, 'title': 'The Artificial Intelligence'},
 {'pageid': 640610, 'ns': 0, 'title': 'Artificial intelligence (AI)'},
 {'pageid': 644536, 'ns': 0, 'title': 'Artificial Intelligence (AI)'},
 {'pageid': 886891, 'ns': 0, 'title': 'Machine understanding'},
 {'pageid': 990276, 'ns': 0, 'title': 'Aretificial intelligence'},
 {'pageid': 1308511, 'ns': 0, 'title': 'Artificial intelligences'},
 {'pageid': 1780695,
  'ns': 0,
  'title': 'The Theory of Artificial Intelligence'},
 {'pageid': 1885131, 'ns': 0, 'title': 'Artificial inteligence'},
 {'pageid': 2436571, 'ns': 0, 'title': 'Artifical Intelligence'},
 {'pageid': 2448119, 'ns':

In [36]:
ID, title, links, redirs = wiki_collect(name)
ID, title, len(links) if links is not None else None, len(redirs) if redirs is not None else None

(1164, 'Artificial intelligence', 1593, 56)

In [34]:
links

[{'ns': 0, 'title': 'ARMM Regional Legislative Assembly'},
 {'ns': 0, 'title': 'Administrative divisions of the Philippines'},
 {'ns': 0, 'title': 'Alaminos, Pangasinan'},
 {'ns': 0, 'title': 'Angeles, Pampanga'},
 {'ns': 0, 'title': 'Antipolo City'},
 {'ns': 0,
  'title': 'Associate Justice of the Supreme Court of the Philippines'},
 {'ns': 0, 'title': 'Bacolod City'},
 {'ns': 0, 'title': 'Bacoor'},
 {'ns': 0, 'title': 'Bago City'},
 {'ns': 0, 'title': 'Baguio'},
 {'ns': 0, 'title': 'Bais City'},
 {'ns': 0, 'title': 'Balanga, Bataan'},
 {'ns': 0, 'title': 'Barangay'},
 {'ns': 0, 'title': 'Barangay councils in the Philippines'},
 {'ns': 0, 'title': 'Batac'},
 {'ns': 0, 'title': 'Batangas City'},
 {'ns': 0, 'title': 'Bayawan'},
 {'ns': 0, 'title': 'Baybay, Leyte'},
 {'ns': 0, 'title': 'Bayugan, Agusan del Sur'},
 {'ns': 0, 'title': 'Bislig'},
 {'ns': 0, 'title': 'Biñan, Laguna'},
 {'ns': 0, 'title': 'Bogo, Cebu'},
 {'ns': 0, 'title': 'Borongan'},
 {'ns': 0, 'title': 'Butuan'},
 {'ns': 0

In [37]:
redirs

[{'pageid': 1268, 'ns': 0, 'title': 'AI'},
 {'pageid': 83979, 'ns': 0, 'title': 'Machine intelligence'},
 {'pageid': 87949, 'ns': 0, 'title': 'AI ethics'},
 {'pageid': 87959, 'ns': 0, 'title': 'AI implications'},
 {'pageid': 246362, 'ns': 0, 'title': 'Artificially intelligent'},
 {'pageid': 524509, 'ns': 0, 'title': 'Artificial Intelligence'},
 {'pageid': 638220, 'ns': 0, 'title': 'The Artificial Intelligence'},
 {'pageid': 640610, 'ns': 0, 'title': 'Artificial intelligence (AI)'},
 {'pageid': 644536, 'ns': 0, 'title': 'Artificial Intelligence (AI)'},
 {'pageid': 886891, 'ns': 0, 'title': 'Machine understanding'},
 {'pageid': 990276, 'ns': 0, 'title': 'Aretificial intelligence'},
 {'pageid': 1308511, 'ns': 0, 'title': 'Artificial intelligences'},
 {'pageid': 1780695,
  'ns': 0,
  'title': 'The Theory of Artificial Intelligence'},
 {'pageid': 1885131, 'ns': 0, 'title': 'Artificial inteligence'},
 {'pageid': 2436571, 'ns': 0, 'title': 'Artifical Intelligence'},
 {'pageid': 2448119, 'ns':